<a href="https://colab.research.google.com/github/nanpolend/machine-learning/blob/master/%E9%A0%90%E6%B8%AC%E6%9D%B1%E4%BA%9E%E5%BC%B7%E9%9C%87%E7%99%BC%E7%94%9F%E6%A9%9F%E7%8E%87%E5%9C%B0%E5%8D%80kaggle%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
東亞地震風險預測系統 v2.0

系統架構：
+-------------------+     +-------------------+     +-------------------+
|  數據獲取層         |     |  特徵工程層        |     |  模型訓練層        |
|  - USGS即時數據    |---->|  - 時空特徵提取    |---->|  - 時間序列交叉驗證 |
|  - 板塊邊界數據     |     |  - 滑動窗口統計    |     |  - 集成學習模型    |
+-------------------+     +-------------------+     +-------------------+
           ↓                                      ↓
+-------------------+     +-------------------+
|  可視化層          |<----|  預測輸出層        |
|  - 風險熱力圖       |     |  - 概率預測        |
|  - 時空分佈圖       |     |  - 閾值判斷        |
+-------------------+     +-------------------+

主要改進：
1. 增強時空特徵工程（滑動窗口+板塊距離）
2. 嚴格時序交叉驗證
3. 自適應顏色映射
4. 動態特徵選擇
"""

import numpy as np
import pandas as pd
import requests
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import roc_auc_score
import folium
from shapely.geometry import Point
import geopandas as gpd
from datetime import datetime, timedelta
from scipy.stats import zscore

class EarthquakePredictor:
    def __init__(self):
        self.quake_data = pd.DataFrame()
        self.plate_data = gpd.GeoDataFrame()
        self.model = RandomForestClassifier(
            n_estimators=200,
            max_depth=10,
            class_weight='balanced_subsample',
            random_state=42
        )
        self.region_bbox = (90, 10, 150, 50)  # 東亞經緯度範圍
        self.feature_window = timedelta(days=90)  # 滑動時間窗口

    def fetch_usgs_data(self, days=365*5):
        """從USGS獲取地震數據（嚴格時序排序）"""
        url = "https://earthquake.usgs.gov/fdsnws/event/1/query"
        params = {
            "format": "geojson",
            "starttime": (datetime.now() - timedelta(days=days)).isoformat(),
            "endtime": datetime.now().isoformat(),
            "minmagnitude": 4.0,
            "maxlatitude": self.region_bbox[3],
            "minlatitude": self.region_bbox[1],
            "maxlongitude": self.region_bbox[2],
            "minlongitude": self.region_bbox[0]
        }

        try:
            response = requests.get(url, params=params, timeout=15)
            response.raise_for_status()

            features = response.json().get('features', [])
            quakes = []
            for f in features:
                props = f.get('properties', {})
                geo = f.get('geometry', {})
                coords = geo.get('coordinates', [])
                if len(coords) >= 2:
                    quakes.append({
                        'time': datetime.utcfromtimestamp(props.get('time', 0)/1000),
                        'lat': coords[1],
                        'lon': coords[0],
                        'mag': props.get('mag', 0),
                        'depth': coords[2] if len(coords)>2 else 0
                    })

            self.quake_data = gpd.GeoDataFrame(
                quakes,
                geometry=[Point(q['lon'], q['lat']) for q in quakes],
                crs="EPSG:4326"
            ).sort_values('time').drop_duplicates(subset=['time', 'lat', 'lon'])

            print(f"獲取地震數據成功，共{len(self.quake_data)}條記錄")
            self._validate_data()

        except Exception as e:
            print(f"數據獲取失敗: {str(e)}")
            raise

    def _validate_data(self):
        """數據質量校驗"""
        if self.quake_data.empty:
            raise ValueError("地震數據為空")
        req_cols = {'time', 'lat', 'lon', 'mag', 'depth', 'geometry'}
        if not req_cols.issubset(self.quake_data.columns):
            missing = req_cols - set(self.quake_data.columns)
            raise ValueError(f"缺少必要字段: {missing}")

    def load_plate_boundaries(self, file_path):
        """加載板塊邊界數據（需提供GeoJSON文件路徑）"""
        try:
            self.plate_data = gpd.read_file(file_path)
            print(f"已加載{len(self.plate_data)}條板塊邊界數據")
            self.plate_data = self.plate_data.to_crs(epsg=4326)
        except Exception as e:
            print(f"板塊數據加載失敗: {str(e)}")
            raise

    def calculate_features(self):
        """計算時空特徵矩陣"""
        if self.quake_data.empty:
            raise ValueError("請先加載地震數據")

        # 空間特徵：板塊距離
        self.quake_data['plate_distance'] = self.quake_data.geometry.apply(
            lambda p: self.plate_data.distance(p).min()
        )

        # 時序滑動特徵
        self.quake_data['time_idx'] = self.quake_data['time'].astype(int)//10**9
        rolling_feats = self.quake_data.set_index('time').sort_index().rolling(
            window=self.feature_window
        ).agg({
            'mag': ['max', 'mean'],
            'depth': 'mean',
            'plate_distance': 'mean'
        })
        rolling_feats.columns = ['_'.join(col).strip() for col in rolling_feats.columns]
        self.quake_data = pd.merge_asof(
            self.quake_data.sort_values('time'),
            rolling_feats.reset_index(),
            on='time'
        )

        # 空間聚類特徵
        self.quake_data['zscore'] = zscore(self.quake_data['mag'])
        print("特徵計算完成，共生成%d個特徵" % (len(self.quake_data.columns)-5))

    def prepare_training_data(self, forecast_window=7):
        """
        準備訓練數據
        :param forecast_window: 預測未來天數
        """
        # 標記目標變量（未來forecast_window天內發生M5+地震）
        self.quake_data['target'] = 0
        time_threshold = self.quake_data['time'] + timedelta(days=forecast_window)

        # 使用空間滑動窗口標記事件
        spatial_buffer = 1.0  # 1度半徑
        for idx, row in self.quake_data.iterrows():
            time_mask = (self.quake_data['time'] > row['time']) & \
                       (self.quake_data['time'] <= time_threshold[idx])
            spatial_mask = self.quake_data.geometry.distance(row.geometry) <= spatial_buffer
            if self.quake_data[time_mask & spatial_mask]['mag'].max() >= 5.0:
                self.quake_data.at[idx, 'target'] = 1

        # 刪除最後forecast_window天的數據以避免未來訊息洩漏
        cutoff = self.quake_data['time'].max() - timedelta(days=forecast_window)
        self.quake_data = self.quake_data[self.quake_data['time'] <= cutoff]

        features = self.quake_data.select_dtypes(include=[np.number]).dropna()
        targets = features.pop('target')
        return features, targets

    def train_model(self, features, targets):
        """使用時序交叉驗證訓練模型"""
        tscv = TimeSeriesSplit(n_splits=5)
        auc_scores = []

        for train_idx, test_idx in tscv.split(features):
            X_train, X_test = features.iloc[train_idx], features.iloc[test_idx]
            y_train, y_test = targets.iloc[train_idx], targets.iloc[test_idx]

            self.model.fit(X_train, y_train)
            preds = self.model.predict_proba(X_test)[:,1]
            auc = roc_auc_score(y_test, preds)
            auc_scores.append(auc)
            print(f"驗證集AUC: {auc:.3f}")

        print(f"平均AUC: {np.mean(auc_scores):.3f} (±{np.std(auc_scores):.3f})")

    def generate_risk_map(self):
        """生成交互式風險地圖"""
        base_map = folium.Map(location=[30, 120], zoom_start=4)

        # 動態顏色映射
        colors = ['#2A9D8F', '#E9C46A', '#F4A261', '#E76F51']
        mag_bins = [4.0, 5.0, 6.0, 7.0, 8.0]

        for idx, quake in self.quake_data.iterrows():
            color_idx = np.digitize(quake['mag'], mag_bins) -1
            folium.CircleMarker(
                location=[quake['lat'], quake['lon']],
                radius=quake['mag']**1.5,
                color=colors[color_idx],
                fill=True,
                tooltip=f"M{quake['mag']:.1f} {quake['time'].date()}"
            ).add_to(base_map)

        return base_map

if __name__ == "__main__":
    predictor = EarthquakePredictor()

    # 數據管道
    predictor.fetch_usgs_data(days=365*3)
    predictor.load_plate_boundaries("plate_boundaries.geojson")  # 需提供實際文件路徑
    predictor.calculate_features()

    # 模型訓練
    features, targets = predictor.prepare_training_data()
    predictor.train_model(features, targets)

    # 可視化
    risk_map = predictor.generate_risk_map()
    risk_map.save("東亞地震風險地圖.html")